# QDK Visualization Tools

This tutorial introduces you to a variety of tools offered by the Microsoft Quantum Development Kit to visualize various elements of Q# programs. 
These tools are extremely helpful both for learners seeking to confirm their understanding of the behavior of the program and for experienced quantum software developers debugging their programs. 

This tutorial covers the following topics:
* Displaying the quantum state of the program: DumpMachine and DumpRegister.
* Displaying the matrix implemented by the quantum operation: DumpOperation.
* Drawing the circuit of an execution path of the program: %trace.
* Running through the quantum program step-by-step: %debug.

# Display the quantum state of a single-qubit program

Let's start with a simple scenario: a program that acts on a single qubit. 
The state of the quantum system used by this program can be represented as a complex vector of length 2, or, using Dirac notation,

$$\begin{bmatrix} \alpha \\ \beta \end{bmatrix} = \alpha|0\rangle + \beta|1\rangle$$

> If you need a refresher on single-qubit quantum systems and their states, please see the tutorial on [the concept of a qubit](../Qubit/Qubit.ipynb).

If this program runs on a physical quantum system, there is no way to get the information about the values of $\alpha$ and $\beta$ at a certain point of the program execution from a single observation. 
You would need to run the program repeatedly up to this point, perform a measurement on the system, and aggregate the results of multiple measurements to estimate $\alpha$ and $\beta$.

However, at the early stages of quantum program development the program typically runs on a simulator - a classical program which simulates the behavior of a small quantum system while having complete information about its internal state. 
You can take advantage of this to do some non-physical things, such as peeking at the internals of the quantum system to observe its exact state without disturbing it!

[DumpMachine](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.diagnostics.dumpmachine) function from [Microsoft.Quantum.Diagnostics namespace](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.diagnostics) allows you to do exactly that. This function is available in standalone Q# applications as well.

### <span style="color:blue">Demo: DumpMachine for single-qubit systems</span>

The following demo shows how to use `DumpMachine` to output the state of the system at any point in the program without affecting the state.

> Note that the Q# code doesn't have access to the output of `DumpMachine`, so you cannot write any non-physical code in Q#!

The output of `DumpMachine` is accurate up to a global phase: sometimes you'll see that all amplitudes are multiplied by some complex number compared to the state you're expecting.

In [1]:
// Run this cell using Ctrl+Enter (⌘+Enter on Mac)
// Then run the next cell to see the output

open Microsoft.Quantum.Diagnostics;

operation SingleQubitDumpMachineDemo () : Unit {
    // This line allocates a qubit in state |0⟩.
    use q = Qubit();
    Message("State |0⟩:");

    // This line prints out the state of the quantum system.
    // Since only one qubit is allocated, only its state is printed.
    DumpMachine();

    // This line changes the qubit to state |+⟩ = (1/sqrt(2))(|0⟩ + |1⟩).
    // 1/sqrt(2) is approximately 0.707107.
    H(q);

    Message("State |+⟩:");
    DumpMachine();

    // This will put the qubit into an uneven superposition,
    // where the amplitudes of |0⟩ and |1⟩ have different absolute values and relative phases.
    Rx(1.0, q);
    Ry(2.0, q);
    Rz(3.0, q);

    Message("Uneven superposition state:");
    DumpMachine();

    // This line returns the qubit to state |0⟩ before releasing it.
    Reset(q);
}

SingleQubitDumpMachineDemo

In [ ]:
%simulate SingleQubitDumpMachineDemo

### <span style="color:blue">Exercise 1</span>: Learn the state of the qubit without measurements

**Input:** A qubit in an unknown state $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$. The amplitudes $\alpha$ and $\beta$ will be real and non-negative.

**Output:** A tuple of two numbers $(\alpha', \beta')$ - your estimates of the amplitudes $\alpha$ and $\beta$.
The absolute errors $|\alpha - \alpha'|$ and $|\beta - \beta'|$ should be less than or equal to 0.001.

The test will call your code exactly once, with the fixed state parameter that will not change if you run the cell several times.

In [4]:
%kata T1_LearnSingleQubitState

operation LearnSingleQubitState (q : Qubit) : (Double, Double) {
    return (0.9689, 0.2474);
}

Success!

# Display the quantum state of a multi-qubit program

Now let's take a look at the general case: a program that acts on $N$ qubits. 
The state of the quantum system used by this program can be represented as a complex vector of length $2^N$, or, using Dirac notation,

$$\begin{bmatrix} x_0 \\ x_1 \\ \vdots \\ x_{2^N-1}\end{bmatrix} = \sum_{k = 0}^{2^N-1} x_k |k\rangle$$

Same as in the single-qubit case, `DumpMachine` allows you to see the amplitudes $x_k$ for all basis states $|k\rangle$ directly.

> Note the use of an integer in the ket notation instead of a bit string with one bit per qubit. 
By default, `DumpMachine` uses little-endian to convert bit strings to integers in the ket notation.
For more details on endiannes, see [the multi-qubit systems tutorial](../MultiQubitSystems/MultiQubitSystems.ipynb#Endianness).

### <span style="color:blue">Demo: DumpMachine for multi-qubit systems</span>

In [5]:
open Microsoft.Quantum.Diagnostics;

operation MultiQubitDumpMachineDemo () : Unit {
    // This line allocates two qubits in state |00⟩.
    use qs = Qubit[2];
    Message("State |00⟩:");

    // This line prints out the state of the quantum system.
    DumpMachine();

    // X gate changes the second qubit into state |1⟩.
    // The entire system is now in state |01⟩, or, in little-endian notation, |2⟩.
    X(qs[1]);
    Message("State |01⟩:");
    DumpMachine();

    CNOT(qs[1], qs[0]);
    Rx(1.0, qs[0]);
    Ry(2.0, qs[1]);
    Rz(3.0, qs[1]);

    Message("Uneven superposition state:");
    DumpMachine();

    // This line returns the qubits to state |0⟩ before releasing them.
    ResetAll(qs);
}

MultiQubitDumpMachineDemo

In [6]:
%simulate MultiQubitDumpMachineDemo

State |00⟩:


|0⟩	1 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0 + 0𝑖
|3⟩	0 + 0𝑖

State |01⟩:


|0⟩	0 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	1 + 0𝑖
|3⟩	0 + 0𝑖

Uneven superposition state:


|0⟩	0.40241210089342777 + 0.028536991480360697𝑖
|1⟩	-0.052236612519413105 + 0.736610409753932𝑖
|2⟩	0.2583858385462908 + -0.01832339151052051𝑖
|3⟩	0.03354074318013797 + 0.47297210492336145𝑖

()

### <span style="color:blue">Exercise 2</span>: Learn the amplitude of a basis state without measurements

**Input:** 2 qubits in an unknown state $|\psi\rangle = \sum_{k = 0}^{3} x_k |k\rangle$. The amplitudes $x_k$ will be real and non-negative.

**Output:** A tuple of two numbers $(x_1', x_2')$ - your estimates of the amplitudes of the state $|1\rangle$ and $|2\rangle$, respectively.
The absolute errors $|x_1 - x_1'|$ and $|x_2 - x_2'|$ should be less than or equal to 0.001.

The test will call your code exactly once, with the fixed state parameter that will not change if you run the cell several times.

In [9]:
%kata T2_LearnBasisStateAmplitudes

operation LearnBasisStateAmplitudes (qs : Qubit[]) : (Double, Double) {
    return (0.3821, 0.3390);
}

Success!

# Configure DumpMachine output

Sometimes you want to focus on certain properties of the quantum state, such as the relative phases of individual basis states, or on certain parts of the state, such as the basis states with larger amplitudes associated with them. 
[%config](https://docs.microsoft.com/qsharp/api/iqsharp-magic/config) magic command allows you to tweak the format of `DumpMachine` output (available in Q# Jupyter Notebooks only). 
Here are some useful options that help you extract the right information:

* `dump.basisStateLabelingConvention` sets the way the basis states are labeled in the output. Here is how the basis state $|10\rangle$ will look like with different settings:

  <table style="background-color: white; border:1px solid; tr  { background-color:transparent; }">
    <col>
    <col width=300>
    <tr>
        <th style="text-align:center; border:1px solid">dump.basisStateLabelingConvention value</th>
        <th style="text-align:center; border:1px solid">DumpMachine output (first two columns)</th>
    </tr>
    <tr>
        <td style="text-align:left; border:1px solid"><tt>"LittleEndian"</tt> (default)</td>
        <td style="text-align:center; border:1px solid"><img src="./img/DumpMachine_LittleEndian.PNG"/></td>
    </tr>
    <tr>
        <td style="text-align:left; border:1px solid"><tt>"BigEndian"</tt></td>
        <td style="text-align:center; border:1px solid"><img src="./img/DumpMachine_BigEndian.PNG"/></td>
    </tr>
    <tr>
        <td style="text-align:left; border:1px solid"><tt>"Bitstring"</tt></td>
        <td style="text-align:center; border:1px solid"><img src="./img/DumpMachine_Bitstring.PNG"/></td>
    </tr>
  </table>


* `dump.truncateSmallAmplitudes`, when set to `true`, removes basis states with measurement probabilities less than a certain threshold from the output. 
By default the measurement probability has to be less than $10^{-10}$ for the state to be truncated; you can change that threshold using the `dump.truncationThreshold` setting.
* `dump.measurementDisplayStyle` and `dump.phaseDisplayStyle` set the style of display for "Measurement Probability" and "Phase" columns; you can choose between bar/arrow, number, both, or none for each of the columns. 
If you choose to display the numeric value of measurement probability, you can configure its probability using `dump.measurementDisplayPrecision`.

### <span style="color:blue">Exercise 3</span>: High-probability basis states

**Input:** $N$ qubits in an unknown state $|\psi\rangle = \sum_{k = 0}^{2^N-1} x_k |k\rangle$. The amplitudes $x_k$ will be real and non-negative.

**Output:** An array of integers which represent the basis states which have amplitudes bigger than 0.01 (in little endian encoding). The integers can be returned in any order.

The test will call your code exactly once, with the fixed state parameter that will not change if you run the cell several times.

In [28]:
%config dump.truncationThreshold = 0.009
%config dump.truncateSmallAmplitudes = false
%config dump.basisStateLabelingConvention = "LittleEndian"
%kata T3_HighProbabilityBasisStates

operation HighProbabilityBasisStates (qs : Qubit[]) : Int[] {
    DumpMachine();
    return [20, 11, 22, 9, 15, 25, 18, 28];
}

|0⟩	0.06159068984682361 + 5.469811143282787E-19𝑖
|1⟩	0.0061590689846823705 + -3.246382187524381E-18𝑖
|10⟩	-2.0007720313290064E-17 + -3.187815940510297E-17𝑖
|2⟩	0.030795344923411804 + -1.4419681984228226E-18𝑖
|20⟩	0.49272551877458953 + 5.871970441988578E-18𝑖
|3⟩	0.0012318137969364786 + 2.320695259140424E-18𝑖
|11⟩	0.18477206954047093 + 4.8759765711564365E-17𝑖
|30⟩	-9.247995015302159E-18 + 1.0180535359841418E-17𝑖
|4⟩	-7.723834734040064E-19 + -1.2305412217148077E-19𝑖
|21⟩	1.5782422919360417E-16 + 4.930035433649776E-18𝑖
|5⟩	0.004311348289277653 + -5.143689074449896E-19𝑖
|12⟩	0.0061590689846824295 + -8.570393281076171E-19𝑖
|31⟩	-2.277746138312189E-34 + 3.9723548673476704E-33𝑖
|6⟩	0.009238603477023538 + -1.84951424988196E-18𝑖
|22⟩	0.1108632417242826 + -1.497548001047319E-17𝑖
|7⟩	0.0018477206954047077 + 1.018872043214307E-19𝑖
|13⟩	0.003695441390809459 + -1.1176182525500324E-18𝑖
|8⟩	0.012318137969364816 + 2.220491799859308E-17𝑖
|23⟩	0.003079534492341196 + 1.3634245276397834E-17𝑖
|9⟩	0.5543162086214128 + 2.58884803079269E-19𝑖
|14⟩	0.0006159068984680611 + 4.951966544023059E-18𝑖
|24⟩	0.007390882781618866 + -3.528173878711018E-17𝑖
|15⟩	0.43113482892776556 + 6.779050315471062E-18𝑖
|25⟩	0.36954413908094247 + -2.963135977609449E-17𝑖
|16⟩	-1.3478681116957138E-19 + -2.521968108167059E-21𝑖
|26⟩	6.15906898468225E-05 + -1.7424319045509376E-18𝑖
|17⟩	0.0006159068984682132 + 1.4656733494813906E-17𝑖
|27⟩	0.0007390882781618709 + -1.9976662157316237E-17𝑖
|18⟩	0.12318137969364733 + -1.8950219978185006E-17𝑖
|28⟩	0.24636275938729485 + -1.1331368386300426E-18𝑖
|19⟩	0.0018477206954047307 + -3.855458873308405E-18𝑖
|29⟩	0.007390882781618896 + 1.3063002781467785E-17𝑖

The returned array has unexpected number of elements
Try again!


In [29]:
open Microsoft.Quantum.Diagnostics;

operation DumpOperationDemo () : Unit {
    DumpOperation(2, ApplyToFirstTwoQubitsCA(CNOT, _));
}

DumpOperationDemo

In [30]:
%simulate DumpOperationDemo

Real:
[[1, 0, 0, 0], 
[0, 0, 0, 1], 
[0, 0, 1, 0], 
[0, 1, 0, 0]]
Imag:
[[0, 0, 0, 0], 
[0, 0, 0, 0], 
[0, 0, 0, 0], 
[0, 0, 0, 0]]

()

> Note that the matrix of the CNOT gate produced by `DumpOperation` in this demo might differ from the one you are used to seeing in quantum computing resources, such as the [multi-qubit gates tutorial](../MultiQubitGates/MultiQubitGates.ipynb#CNOT-Gate):
> $$\begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{bmatrix}$$
>
> Similarly to `DumpMachine`, the way `DumpOperation` evaluates the matrix depends on the conventions used by the simulator. 
> Q# full state simulator tends to use little-endian encoding for converting basis states to integers, and `DumpOperation` uses the same convention for converting basis states to the indices of matrix elements.
> Thus, the second column of the CNOT matrix corresponds to the input state $|1\rangle_{LE} = |10\rangle$, which the CNOT gate converts to $|11\rangle = |3\rangle_{LE}$, meaning that the last element is going to be $1$.
>
> A lot of resources on quantum computing use big-endian encoding, so in them the second column of the CNOT matrix corresponds to the input state $|1\rangle_{BE} = |01\rangle$, which should be left unchanged by the CNOT gate.

### <span style="color:blue">Exercise 4</span>: Read the operation matrix

In this task you will be interpreting the matrix of an operation `MysteryOperation1` that implements a unitary operation $U$, defined in the `Quantum.Kata.VisualizationTools` namespace. 
This operation will not be passed as an input to your task; you can access it by creating a separate code cell with a new operation (or modifying `DumpOperationDemo`), and using `%simulate` for that operation.

**Input:** None.

**Output:** A single floating-point number: the amplitude of the $|10\rangle$ basis state in the state $U|01\rangle$ (i.e., the element of the matrix implemented by $U$ which describes the transformation $|01\rangle \rightarrow |10\rangle$). The result must be within 0.001 from the actual value.

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
  There are two ways to do this task. You can use DumpOperation directly, or (since you're looking for just one coefficient, not all of them) you can allocate 2 qubits, prepare them in the right basis state, apply the MysteryOperation1 and use DumpMachine to see the amplitudes of the resulting superposition state.
</details>

In [ ]:
%kata T4_ReadOperationMatrix

operation ReadMysteryOperationMatrix () : Double {
    // ...
}

# Display the circuit performed by the operation

[Quantum circuits](https://en.wikipedia.org/wiki/Quantum_circuit) are a very common way of visualizing quantum programs; you'll see them in a lot of tutorials, papers and books on quantum computing.
Quantum circuits are a less powerful way of expressing the quantum computation compared to a quantum program. 
They don't offer a good way to show the values of classical variables and their evolution, the decisions made based on the classical parameters or the measurement results, or even flow control structures such as loops or conditional statements.
At the same time, they can be convenient to get a quick idea of what the program did, or to compare your implementation to the one offered in a book.

[`%trace` magic command](https://docs.microsoft.com/qsharp/api/iqsharp-magic/trace) (available only in Q# Jupyter Notebooks) offers a way to trace one run of the Q# program and to build a circuit based on that execution.
Note that these circuits include only the quantum gates executed by the program. 
They might differ in different runs of the same program, if that program takes parameters, has conditional branching or other behaviors that can change the sequence of gates applied by the program.

You can pass parameters to the operation traced with `%trace` same as for `%simulate`, by adding `<parameterName>=<value>` for each parameter after the operation name.


### <span style="color:blue">Demo: Using %trace</span>

Let's take a look at the circuit produced by the `MultiQubitDumpMachineDemo` operation defined earlier.

> If the cell below gives you an "Invalid operation name" error, return to the demo [DumpMachine for multi-qubit systems](#Demo:-DumpMachine-for-multi-qubit-systems) and run it to define the operation.

Try clicking on the gates that show a magnifying glass when the cursor hovers over them to see their internal implementation.

In [31]:
%trace MysteryOperation2 N=3

DisplayableHtmlElement { Html = <div id='execution-path-container-5ab3ca8f-f4fa-41de-95d5-1e88cf4d1681' /> }

### <span style="color:blue">Exercise 5</span>: Read the trace of the operation

In this task you will be interpreting the trace of an operation `MysteryOperation2` that implements a certain quantum algorithm. 
This operation will not be passed as an input to your task; you can access it by creating a separate code cell with a new operation or using `%trace` for that operation directly.

`MysteryOperation2` takes a single integer named $N$ as an input.

**Input:** None.

**Output:** A tuple of 3 integers that describe the behavior of `MysteryOperation2` for its input $N = 3$:

1. The number of qubits used by the operation (the "width" of the circuit).
2. The number of Hadamard gates used by it (including adjoint Hadamard gates, which are the same as Hadamard gates).
3. The number of CNOT gates used by it.

In [ ]:
%kata T5_ReadOperationTrace

operation ReadMysteryOperationTrace () : (Int, Int, Int) {
    // ...
}

# Run through the program execution gate by gate

Finally, [`%debug` magic command](https://docs.microsoft.com/qsharp/api/iqsharp-magic/debug) (available only in Q# Jupyter Notebooks) allows you to combine tracing the program execution (as a circuit) and observing the program state at the same time.

### <span style="color:blue">Demo: Using %debug</span>

> You cannot switch to executing a different notebook cell in the middle of the debug session.
If you need to stop before it's complete, you can use `Kernel -> Interrupt` menu item.

In [ ]:
%debug MysteryOperation2 N=2

## Conclusion

We hope you've learned a lot from this tutorial.
With these tools you're well equipped to continue your quantum computing journey!